In [1]:
from pyspark.sql import SparkSession
# from pyspark.ml.regression import LogisticRegression

In [2]:
spark = SparkSession.builder.appName('titanic').getOrCreate()
df = spark.read.csv('/FileStore/tables/titanic.csv',header=True,inferSchema=True)
# could also read data in with:
# --> `df = spark.sql("select * from titanic_csv")`

In [3]:
df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [4]:
df.columns

Out[ 4 ]: 
['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [5]:
my_cols = df.select(['Survived',
                    'Pclass',
                    'Sex',
                    'Age',
                    'SibSp',
                    'Parch',
                    'Fare',
                    'Embarked'])

In [6]:
# drop missing data from the columns
my_final_data = my_cols.na.drop()

In [7]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [8]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
# now OneHotEncode this--transforms actual numbers for categories into OneHotEncoding
# A B C
# 0 1 2
# One Hot Encode
# KEY A B C
# [1, 0, 0] --> encodes the key 'A' so that it can be binary values
# [0, 1, 0] --> encodes key 'B'
gender_index_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [9]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [10]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],
                           outputCol='features')

In [11]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [12]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [13]:
pipeline = Pipeline(stages=[gender_indexer,
                            embark_indexer,
                            gender_index_encoder,
                            embark_encoder,
                            assembler,
                            log_reg_titanic])

In [14]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

In [15]:
# call pipeline object just like you would a ML algo
fit_model = pipeline.fit(train_data)

In [16]:
results = fit_model.transform(test_data)

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [18]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [19]:
results['Survived','prediction'].show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+--------+----------+
only showing top 20 rows

In [20]:
# measure area under the curve (AUC)
AUC = my_eval.evaluate(results)

In [21]:
AUC

Out[ 35 ]: 0.755952380952381